In [1]:
import torch
import os
from torchvision.datasets import CIFAR10
from linear_classifier import LinearClassifier

In [2]:
# Dataset Download
download = not os.path.isdir("cifar-10-batches-py")
dset_train = CIFAR10(root=".", download=download, train=True)

X = torch.tensor(dset_train.data, dtype=torch.float32).permute(0, 3, 1, 2).div_(255)
y = torch.tensor(dset_train.targets, dtype=torch.int64)

X.cuda()
y.cuda()


tensor([6, 9, 9,  ..., 9, 1, 1], device='cuda:0')

In [3]:
# Dataset Preprocessing

# Flatting
X = X.reshape(X.shape[0], -1)

# Adding Bias
ones_train = torch.ones(X.shape[0], 1, device=X.device)
X = torch.cat([X, ones_train], dim=1)


# Validation and Training Split
validation_ratio = 0.3
num_training = int(X.shape[0] * (1.0 - validation_ratio))
num_validation = X.shape[0] - num_training
X_train = X[num_training : num_training + num_validation]
y_train = y[num_training : num_training + num_validation]

X_val = X[0:num_training]
y_val = y[0:num_training]

In [5]:
# Classify

classifier = LinearClassifier()
classifier.train(X_train, y_train, X_val, y_val, 1e-2,1e-2,2000)



KeyboardInterrupt: 